In [35]:
from datasets import load_dataset
c4_su = load_dataset("allenai/c4", "su")
c4_su_dev_dict = c4_su.data['validation'].to_pydict()

In [49]:
import pandas as pd
mabl_su_df = pd.read_csv('../langdata/su.csv')
# mabl_su_df = pd.read_csv('../translate-test/su.csv')
# mabl_su_df = pd.read_csv('../data/syntax_chunked/syntax_tagged_su.csv')

In [50]:
mabl_su_df

,ending1,ending2,startphrase,labels
0,hirup kawas kakekang,hirup kawas bebas.,hirup kawas kurung batok,0
1,hirup kawas kakekang,hirup kawas bebas.,hirup kawas murag bulu bitis.,1
2,jadi jalma teh kudu rajin,jadi jalma teh kudu kedul.,jadi jalma teh kudu hampang birit,0
3,jadi jalma teh kudu rajin,jadi jalma teh kudu kedul.,jadi jalma teh kudu beurat birit.,1
4,mamat lumpat siga gancang,mamat lumpat siga laun.,Mamat lumpatna siga uncal,0
...,...,...,...,...
595,Balik sareng mantan mah seru,Balik sareng mantan mah bosen.,Balik deui sareng mantan sapertos nganggo baju...,1
596,Hubungan jarak jauh pinuh ku rintangan,Hubungan jarak jauh teu aya rintanganana.,Hubungan jarak jauh ibarat arum jeram di sungai,0
597,Hubungan jarak jauh pinuh ku rintangan,Hubungan jarak jauh teu aya rintanganana.,Hubungan jarak jauh ibarat numpakan kano di ta...,1
598,Panghina nyieun leuwih getol,Panghina nyieun nelangsa.,Panghina ti batur mah siga kusir keur mecut kuda,0


In [56]:
start_phrases = mabl_su_df['ending1'].tolist()


In [54]:
c4_su_toy = c4_su_dev_dict['text']

In [55]:
c4_su_dev_dict['text'][:5]

['Etas luctus - Linda Naor\nEtas luctus2014-08-212014-08-21https://lindanaor.com/wp-content/uploads/2019/01/linda-naor-logo.pngLinda Naorhttps://lindanaor.com/wp-content/uploads/2019/01/linda-naor-logo.png200px200px',
 'Lukas 9 (MANGGARAI)\nMori Yésus Wuat Campulu Suad Rasuln (Mat 10:5-15; Mr 6:7-13)\n1Itu kali Mori Yésus bénta campulu suad rasul, poli hitu téing mberes agu kuasa oné isé, kudut wur jing, agu latang te ina taung sanggéd beti.\n2Agu hia wuat isé kudut keréba Adak de Mori Keraéng, agu pandé ina ata,\n3mai taé Diha ngong isé: “Néka ba apa-apa du ngo lako, néka ba do’ar ko bokong, roti ko doi, ko suan baju.\n4Méu émé poli tiba lata oné ca mbaru, ka’éng nitum dengkir méu angkak nitu mais.\n5Émé manga ata situt toé gorid te tiba méu, pé’ang oné mai béndar disés, agu péntang kebok wa’is, cama ného te langi isé.”\n6Itu kali ngod isé, agu isé lako labok taung béo, cang agu wéro keréba di’ad agu pandé ina ata beti néténg tana.\nHi Hérodés Agu Mori Yésus (Mat 14:1-12; Mr 6:14-29)\

### matching

In [58]:
queries = start_phrases[:5]
example_pool = c4_su_dev_dict['text'][:10]

In [73]:
import nltk

def calculate_ngram_overlap(text, query, n=2):
  """
  Calculates the n-gram overlap between a text and a query.
  """
  text_ngrams = set(nltk.ngrams(text.split(), n))
  query_ngrams = set(nltk.ngrams(query.split(), n))
  overlap = len(text_ngrams & query_ngrams) / len(query_ngrams)
  return overlap

# Example usage
corpus = ["This is a sample text.", "Another example sentence.", "This text is unrelated."]
queries = ["sample text", "unrelated topic"]

for query in queries:
  relevant_texts = [text for text in corpus if calculate_ngram_overlap(text, query) > 0.1]
  print(f"Relevant texts for query '{query}': {relevant_texts}")

Relevant texts for query 'sample text': []
Relevant texts for query 'unrelated topic': []


In [82]:
calculate_ngram_overlap("hi", "hi this is", 1)

0.3333333333333333

### try langchain

In [130]:
from langchain.prompts.example_selector.ngram_overlap import ngram_overlap_score
import numpy as np
 
# based on langchain
def select_examples(query: str, example_pool: list[str], threshold = -1.0) -> list[dict]:
    selected_examples = []
    k = len(example_pool)
    score = [0.0] * k

    for i in range(k):
        score[i] = ngram_overlap_score(
            [query], [example_pool[i]]
        )

    while True:
        arg_max = np.argmax(score)
        if (score[arg_max] < threshold) or abs(
            score[arg_max] - threshold
        ) < 1e-9:
            break

        selected_examples.append(example_pool[arg_max])
        score[arg_max] = threshold - 1.0
    print(score)

    return selected_examples

select_examples(
    query = "Spot can run fast.", 
    example_pool = ["See Spot run.", "My dog barks.", "Spot can run."],
    threshold=0.001
)

[-0.999, 0.0, -0.999]


['Spot can run.', 'See Spot run.']